In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator
from shared.analyzers import FixedEffectsAnalyzer
import matplotlib.pyplot as plt

In [2]:
N = 50
T = 100
alpha = 0.05

no_eff = ConstantEffectSimulator("No Effect", N, 0., 0., 0.)
shared_random_eff = ConstantEffectSimulator("Shared Random Effect", N, 0., 1., 0.)
distinct_random_eff = ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1.)
noisy_distinct_random_eff = ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1., epsilon_sd=1.)
moderated_eff = ModeratedEffectSimulator("Moderated Effects", N, 0., 1., 1., 0.5, 0.2, epsilon_sd=1.)
effect_simulators = [no_eff, shared_random_eff, distinct_random_eff, noisy_distinct_random_eff, moderated_eff]

fixed_effects = FixedEffectsAnalyzer("Fixed Effects", alpha)
analyzers = [fixed_effects]

for i, effect_simulator in enumerate(effect_simulators):
    effect_simulator.sample_params()
    data_simulator = DataSimulator(N, T).simulate(effect_simulator)
    for j, analyzer in enumerate(analyzers):
        results = analyzer.analyze(data_simulator)

In [5]:
results.get_mean_series()

mean_estimate    0.014693
mean_lower       0.014663
mean_upper       0.014722
dtype: float64